## Exploring how word embedding of the Bert model are related - consecutive word embeddings in each layer and same word embedding of current and the first layer

In [1]:
from transformers import BertTokenizer, BertModel
import torch

/data/bdlml/mkabra/mls_python_venv/miniconda/envs/mk_ve_2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-24 11:36:25.006402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-24 11:36:25.220668: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-24 11:36:26.227666: W tensorflow/compiler/xla

In [2]:
bert_uncased="./models--bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(bert_uncased)
model = BertModel.from_pretrained(bert_uncased)


Some weights of the model checkpoint at /data/bdlml/mkabra/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Adding the position of each word in the tokenized text.

Tokenizing the text and decoding it then

In [3]:


text="card member complained about the banking services "# express"
encoded_input = tokenizer(text, return_tensors='pt')
output1 = model(**encoded_input, output_hidden_states=True)
print('encoded_input', encoded_input['input_ids'].shape)
print('last_hidden_state shape ',output1['last_hidden_state'].shape)
#print({x : tokenizer.encode(x, add_special_tokens=False) for x in text.split()})

decoded=[]
encoded=encoded_input['input_ids'][0].detach().numpy()
for i in range(len(encoded)):
    id_=encoded[i]
    decoded.append([i , id_, ''.join(tokenizer.decode(id_).split(' ')) ])
print('decoded ',decoded)

encoded_input torch.Size([1, 9])
last_hidden_state shape  torch.Size([1, 9, 768])
decoded  [[0, 101, '[CLS]'], [1, 4003, 'card'], [2, 2266, 'member'], [3, 10865, 'complained'], [4, 2055, 'about'], [5, 1996, 'the'], [6, 8169, 'banking'], [7, 2578, 'services'], [8, 102, '[SEP]']]


Comparing the similarity of two  consecutive words ('banking' & 'services') in each layer

#Similarity increases with each layer

In [4]:


cos_sim = torch.nn.CosineSimilarity(dim=0) 


def cos_sim_b_w_words_hiddenstate(elem1_pos, elem2_pos):
    print('Elem1 ', decoded[elem1_pos] )
    print('Elem2', decoded[elem2_pos] )
    
    for i in range(len(output1['hidden_states'])):
        elem1_embed=output1['hidden_states'][i][0][elem1_pos]
        elem2_embed=output1['hidden_states'][i][0][elem2_pos]
        print('Layer no. ', i ,'similarity b/w elements', cos_sim(elem1_embed, elem2_embed).detach().numpy() ) 
    print('\n')


cos_sim_b_w_words_hiddenstate(6,7)

#Similarity increases with each layer
#comparing the similarity of two  consecutive words ('card' & 'member')
cos_sim_b_w_words_hiddenstate(1,2)

Elem1  [6, 8169, 'banking']
Elem2 [7, 2578, 'services']
Layer no.  0 similarity b/w elements 0.144236
Layer no.  1 similarity b/w elements 0.29189572
Layer no.  2 similarity b/w elements 0.29617742
Layer no.  3 similarity b/w elements 0.2963208
Layer no.  4 similarity b/w elements 0.315475
Layer no.  5 similarity b/w elements 0.34980845
Layer no.  6 similarity b/w elements 0.35394484
Layer no.  7 similarity b/w elements 0.3901196
Layer no.  8 similarity b/w elements 0.4002793
Layer no.  9 similarity b/w elements 0.45090446
Layer no.  10 similarity b/w elements 0.5217607
Layer no.  11 similarity b/w elements 0.6460583
Layer no.  12 similarity b/w elements 0.6108783


Elem1  [1, 4003, 'card']
Elem2 [2, 2266, 'member']
Layer no.  0 similarity b/w elements 0.12738381
Layer no.  1 similarity b/w elements 0.2428741
Layer no.  2 similarity b/w elements 0.2686418
Layer no.  3 similarity b/w elements 0.27650452
Layer no.  4 similarity b/w elements 0.31683552
Layer no.  5 similarity b/w elements

Computes cosine similarity of the 1st hidden state with the subsequent hidden states (of a word)

The similarity will decrease with each layer

In [5]:
def cos_sim_word_hiddenstate(elem_pos):
    #Computes cosine similarity of the 1st hidden state with the subsequent hidden states (of a word)
    #The similarity will decrease with each layer
    print('Elem ', decoded[elem_pos] )
    
    elem_hiddenstate0_emebd=output1['hidden_states'][0][0][elem_pos]
    
    for i in range(1, len(output1['hidden_states'])):
        elem_hiddenstate_N_emebd=output1['hidden_states'][i][0][elem_pos]
        print('Hidden state ', i ,'similarity', cos_sim(elem_hiddenstate0_emebd, elem_hiddenstate_N_emebd) )
    print('\n')
    
    
cos_sim_word_hiddenstate(6)   

Elem  [6, 8169, 'banking']
Hidden state  1 similarity tensor(0.8465, grad_fn=<SumBackward1>)
Hidden state  2 similarity tensor(0.7717, grad_fn=<SumBackward1>)
Hidden state  3 similarity tensor(0.7011, grad_fn=<SumBackward1>)
Hidden state  4 similarity tensor(0.6403, grad_fn=<SumBackward1>)
Hidden state  5 similarity tensor(0.5595, grad_fn=<SumBackward1>)
Hidden state  6 similarity tensor(0.5159, grad_fn=<SumBackward1>)
Hidden state  7 similarity tensor(0.4896, grad_fn=<SumBackward1>)
Hidden state  8 similarity tensor(0.4522, grad_fn=<SumBackward1>)
Hidden state  9 similarity tensor(0.4168, grad_fn=<SumBackward1>)
Hidden state  10 similarity tensor(0.3797, grad_fn=<SumBackward1>)
Hidden state  11 similarity tensor(0.3096, grad_fn=<SumBackward1>)
Hidden state  12 similarity tensor(0.2936, grad_fn=<SumBackward1>)


